In [6]:
from dotenv import load_dotenv
load_dotenv('.env-db')
from utils.utils import get_database, all_scales
from transformers import LlamaTokenizerFast
tokenizer = LlamaTokenizerFast.from_pretrained('hf-internal-testing/llama-tokenizer')
import pandas as pd
def count_tokens(prompt):
    return len(tokenizer.tokenize(prompt))

## Copy existing reference DB to new one

In [13]:
db = get_database()
llms = ['OpenAssistant/llama2-13b-orca-8k-3319', 'openchat/openchat_v3.1', 'baichuan-inc/Baichuan-13B-Base']
# llm_name = 'OpenAssistant/llama2-13b-orca-8k-3319'
# llm_name = openchat/openchat_v3.1
# llm_name = 'baichuan-inc/Baichuan-13B-Base'
dim = 'contrast'
# all_ratings_llm = pd.DataFrame(list(db[f'queries/{llm_name}/{dim}'].find({})))
# all_ratings_llm = pd.DataFrame(list(db[f'queries/{dim}/{llm_name}'].find({})))

In [15]:
####### WARNING: THIS WILL REPLACE ALL QUERIES IN THE COLLECTION ####################
############ for llm_name in llms:
############     reference_queries = list(db[f'queries/reference/{dim}'].find({'tot_improved': False})) ##########
############     # print(len(reference_queries))/
############     db[f'queries/{llm_name}/{dim}'].insert_many(reference_queries) ###########
# db.collection.create_index([(<key and index type specification>)], <options> )

In [22]:
# db[f'core_prompts/{dim}'].update_many({"Prompt": -100}, {"$set": {'rating': -1}})
all_core_prompts = {}
all_core_prompts_adjusted = {}
for dim in all_scales:
    if dim != 'contrast':
        all_core_prompts[dim] = list(db[f'core_prompts/{dim}'].find({}))
        all_core_prompts_adjusted[dim] = []
        for prompt_doc in all_core_prompts[dim]:
            if 'Prompt' in prompt_doc:
                prompt_doc['combined_prompt'] = prompt_doc['Prompt']
                prompt_doc.pop('Prompt')
            else:
                prompt_doc['tot_improved'] = True
                prompt_doc.pop('dimension')
            all_core_prompts_adjusted[dim].append(prompt_doc)


In [23]:
for dim in all_core_prompts_adjusted:
    db[f'core_prompts/{dim}'].drop()
    db[f'core_prompts/{dim}'].insert_many(all_core_prompts_adjusted[dim])

In [ ]:
# len(all_queries)
import requests
requests.post("http://localhost:8000/calibrations/cleanup", data="{}")

In [ ]:
# db[f'queries/{llm_name}/{dim}'].insert_many(all_queries)

In [ ]:
# get average latency
latencies = list(db[f'queries/{llm_name}/{dim}'].find({"latency":{"$gt":0}}))
latencies = pd.DataFrame(latencies)
latencies['latency'].hist(bins=100)
print(f"Average latency: {sum(latencies['latency'])/len(latencies['latency'])}")

In [ ]:
latencies['rating'] = latencies['rating'].astype(int)
latencies[(0 <= latencies['rating']) & (latencies['rating'] <= 10)]['rating'].hist(bins=10)

In [ ]:
db[f'queries/{llm_name}/{dim}'].update_many({"latency":{"$gt":0}, "rating": -1}, {"$set": {"latency": -1, "num_tries": 0}})

In [ ]:
count_tokens('10')

In [ ]:
# import tqdm
# # display dist. of prompt lengths
# dim = 'contrast'
# prefixes = {prefix['prefix_index']:prefix for prefix in list(db['prefixes'].find({}))}
# prompts = {prompt['prompt_index']:prompt for prompt in list(db[f'core_prompts/{dim}'].find({}))}
# samples = {sample['sample_index']:sample for sample in list(db[f'samples/{dim}'].find({}))}
# tokens_dist = []
# prefix = max(prefixes.values(), key=lambda x : len(x['prefix']))
# for prompt in tqdm.tqdm(prompts):
#     for sample in samples:
#         t = count_tokens(prefix['prefix'] + prompts[prompt]['combined_prompt'] + samples[sample]['sample'])
#         tokens_dist.append(t)
# pd.Series(tokens_dist).hist(bins=100)

In [12]:
import tqdm

skip_prefix_severe = ['reciprocity', 'unity', 'scarcity']
for dim in all_scales:
    query_docs = []
    print("UPLOADING DIMENSION", dim)
    all_dim_prompts = list(db[f'core_prompts/{dim}'].find({}))
    all_dim_samples = list(db[f'samples/{dim}'].find({}))
    all_prefixes = list(db[f'prefixes'].find({}))
    for prefix_doc in tqdm.tqdm(all_prefixes):
        prefix_ind = prefix_doc['prefix_index']
        if prefix_ind == 2 and dim in skip_prefix_severe:
            continue
        for prompt_doc in all_dim_prompts:
            for sample_doc in all_dim_samples:
                query_doc = {
                    'prefix_index': prefix_doc['prefix_index'],
                    'prompt_index': prompt_doc['prompt_index'],
                    'sample_index': sample_doc['sample_index'],
                    'tot_improved': False if 'tot_improved' in prompt_doc else True,
                    'dimension': dim,
                    'latency': -1.0,
                    'num_tries': 0,
                    'rating': -1,
                }
                if not query_doc['tot_improved']:
                    query_docs.append(query_doc)
    db[f'queries/reference/{dim}'].insert_many(query_docs)

UPLOADING DIMENSION contrast


100%|██████████| 6/6 [00:00<00:00, 23.89it/s]


UPLOADING DIMENSION authority


100%|██████████| 6/6 [00:00<00:00, 19.95it/s]


UPLOADING DIMENSION consensus


...

100%|██████████| 6/6 [00:00<00:00, 16.13it/s]


UPLOADING DIMENSION liking


100%|██████████| 6/6 [00:00<00:00, 15.92it/s]


UPLOADING DIMENSION reciprocity


100%|██████████| 6/6 [00:00<00:00, 11.41it/s]
